In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, BertModel
from torch import nn
import torch

/home/pico/myCodes/medfusion/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("StanfordAIMI/RadBERT")
# model = AutoModelForMaskedLM.from_pretrained("StanfordAIMI/RadBERT")
# model = AutoModel.from_pretrained("StanfordAIMI/RadBERT")
model = BertModel.from_pretrained("StanfordAIMI/RadBERT")

Some weights of the model checkpoint at StanfordAIMI/RadBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
input_str = "A photo of a lung xray with a visible pleural effusion"
inputs = tokenizer(input_str, return_tensors="pt")
outputs = model(**inputs)

In [30]:
print(inputs['input_ids'][0])
print(tokenizer.decode(inputs['input_ids'][0]))

tensor([  101,   170,  6307,  1104,   170, 13093,   193,  6447,  1114,   170,
         5085,   185,  1513, 12602,   174,  3101, 17268,   102])
[CLS] a photo of a lung xray with a visible pleural effusion [SEP]


In [53]:
c = outputs.pooler_output
# c = outputs.last_hidden_state

In [54]:
c.shape

torch.Size([1, 18, 768])

In [60]:
# emb_layer = nn.Embedding(768, 1024)

emb_layer = nn.Sequential(
    nn.Linear(768, 1024),
    nn.LayerNorm(1024),
    nn.ReLU(),
    nn.Linear(1024, 1024),
    nn.LayerNorm(1024),
)

In [61]:
final_output = emb_layer(c)

In [62]:
final_output.size()

torch.Size([1, 18, 1024])

In [63]:
final_output

tensor([[[ 1.5708, -0.7867,  0.3695,  ...,  1.0730, -0.4946, -0.1208],
         [ 0.4086, -0.8935,  0.2968,  ...,  1.8416,  1.0497, -1.1519],
         [ 0.7235,  0.0836, -0.9768,  ..., -0.2204, -0.5774, -1.7679],
         ...,
         [ 1.4654, -0.3428,  0.3720,  ...,  0.7125, -0.4923, -0.9060],
         [ 2.0268,  0.0408, -1.0733,  ...,  0.1879, -0.1402, -1.8658],
         [ 1.5708, -0.7867,  0.3695,  ...,  1.0730, -0.4946, -0.1208]]],
       grad_fn=<NativeLayerNormBackward0>)